In [1]:
# Running the Colab Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Importing of the necessary libraries and the modules

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
!pip install xgboost
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import roc_auc_score

from xgboost.sklearn import XGBClassifier


#Loading data

In [3]:
data = pd.read_csv('/content/drive/MyDrive/Home Credit/home-credit-default-risk/dropped_train.csv').sample(frac =.50)
data

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_EMP_PHONE,FLAG_WORK_PHONE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,PHONE,WRONG_ADRESS,DOCUMENT_SUM_1,DOCUMENT_SUM_2,BUILDING_FEATURES,NAME_CONTRACT_TYPE,CODE_GENDER,...,client_installments_AMT_INSTALMENT_sum_min,client_installments_AMT_INSTALMENT_max_max,client_installments_AMT_PAYMENT_max_max,client_installments_AMT_PAYMENT_sum_mean,client_installments_AMT_INSTALMENT_sum_mean,client_installments_AMT_PAYMENT_min_sum,client_installments_AMT_INSTALMENT_min_sum,client_installments_AMT_PAYMENT_sum_max,client_installments_AMT_INSTALMENT_sum_max,client_installments_AMT_PAYMENT_mean_sum,client_installments_AMT_INSTALMENT_mean_sum,client_installments_AMT_INSTALMENT_max_sum,client_installments_AMT_PAYMENT_max_sum,client_installments_AMT_PAYMENT_sum_sum,client_installments_AMT_INSTALMENT_sum_sum,credit_power1,credit_power2,credit_power3,credit_power4,credit_power5,credit_power6,credit_power7,credit_power8,CHILDREN_TO_FAMILY,CREDIT_TO_INCOME,CREDIT_TERM,ANNUITY_TO_INCOME,GOODS_PRICE_TO_CREDIT,INCOME_PER_CHILD,RATE_EMPLOYED,RATE_30_SOCIAL,RATE_60_SOCIAL,YEARS_EMPLOYED,YEARS_BIRTH,INTEREST,INTEREST_RATE,INTEREST_SHARE,income_balance,income_regularity,balance_regularity
202498,0.0,0.0,112500.0,298467.0,35419.5,283500.0,0.007114,27.19,-1046.0,-4048.0,-2504.0,0.0,1.0,0.0,1.0,2.0,2.0,15.0,0.0000,0.2634,0.2637,0.0000,0.0,0.0,0.0,0.0,-349.0,0.0,0.0,0.0,0.0,0.0,4.0,2.0,2.0,0.0000,0.25,2.639,0,1,...,35038.080,3185.280,3185.280,35038.080,35038.080,35038.08,35038.08,35038.080,35038.080,35038.08,35038.08,35038.08,35038.08,3.854189e+05,3.854189e+05,20.869068,inf,42.750058,inf,inf,20.869068,inf,inf,0.0000,2.652,8.426629,0.31500,0.9497,inf,-38.470,NaN,NaN,2.865,-0.07446,126567.0,0.7827,0.4240,NaN,NaN,NaN
130684,0.0,0.0,225000.0,640764.0,20799.0,459000.0,0.072500,64.56,0.0,-4320.0,-4320.0,0.0,0.0,0.0,2.0,1.0,1.0,11.0,0.0000,0.7695,0.0000,0.3820,7.0,0.0,7.0,0.0,-2856.0,0.0,0.0,0.0,6.0,0.0,5.0,3.0,0.0,0.0625,0.00,11.410,0,0,...,134996.720,48196.844,48196.844,651245.300,765414.400,1236278.40,3794366.20,1456356.900,1535909.600,3358647.50,3874697.20,3878971.00,3878971.00,1.022455e+08,1.201701e+08,5.834609,inf,311.792128,inf,inf,145.276790,inf,inf,0.0000,2.848,30.807443,0.09247,0.7163,inf,0.000,0.0000,0.0,-0.000,-0.17690,4101408.0,0.6710,6.4020,NaN,NaN,NaN
47770,0.0,0.0,90000.0,270045.0,28485.0,256500.0,0.035800,49.53,-1062.0,-6250.0,-1609.0,0.0,1.0,0.0,2.0,2.0,2.0,10.0,0.5630,0.6190,0.8050,0.0000,4.0,0.0,4.0,0.0,-693.0,0.0,0.0,0.0,0.0,0.0,5.0,2.0,2.0,0.0000,0.25,2.639,0,0,...,104024.200,133281.580,133281.580,272256.720,244084.530,828678.25,270572.90,341671.060,292022.160,1021380.30,921661.75,1829194.20,1835946.90,6.261904e+06,5.613944e+06,6.961905,3.714286,inf,inf,inf,inf,inf,inf,0.0000,3.000,9.480253,0.31640,0.9497,inf,-21.440,0.0000,0.0,2.910,-0.13570,584505.0,1.6760,2.1640,NaN,NaN,NaN
154656,0.0,1.0,337500.0,675000.0,76473.0,675000.0,0.026400,36.16,-975.0,-2484.0,-4404.0,31.0,1.0,0.0,3.0,2.0,2.0,18.0,0.0000,0.7783,0.6514,0.0525,3.0,1.0,3.0,0.0,-2010.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,2.0,0.0625,0.00,2.729,0,1,...,38149.336,45616.140,74658.330,288512.970,197892.920,330294.75,330294.75,589258.500,388599.840,718521.10,517862.44,743015.20,1215137.10,8.943902e+06,6.134680e+06,3.347935,1.867463,6.493798,3.852333,1.867463,3.347935,4.736725,3.104119,0.3333,2.000,8.826645,0.22660,1.0000,337500.0,-26.970,0.3333,0.0,2.672,-0.09906,10795950.0,2.5430,15.9900,NaN,NaN,NaN
266524,0.0,1.0,270000.0,746280.0,54436.5,675000.0,0

#Train Test Split

In [ ]:
X = data.drop(['TARGET'], axis=1)
y = data['TARGET']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state =42)


In this problem, the data is imbalanced. So we can’t use accuracy as a error metric. When data is imbalanced we can use Log loss, F1-score and AUC. Here we are sticking to AUC which can handle imbalanced datasets.

Due to the high unbalance between 0 and 1 target flags, we set  the scale pos weight parameter  to the ratio between the number of 0s and the number of 1s. 

Area Under Curve (AUC)

An ROC curve is the most commonly used way to visualize the performance of a binary classifier, and AUC is (arguably) the best way to summarize its performance in a single number.

In [ ]:
ratio = (y == 0).sum()/ (y == 1).sum()
ratio

11.447862694300518

#Model XGBoost

In [ ]:
# fit xgboost on an imbalanced classification dataset
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
clf = XGBClassifier(n_estimators=1000, objective='binary:logistic', gamma=0.1, subsample=0.5, scale_pos_weight=ratio )
clf.fit(X_train, y_train, eval_set=[(X_test, y_test)], eval_metric='auc', early_stopping_rounds=10)

[0]	validation_0-auc:0.681304
Will train until validation_0-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.692027
[2]	validation_0-auc:0.695133
[3]	validation_0-auc:0.70142
[4]	validation_0-auc:0.704452
[5]	validation_0-auc:0.70766
[6]	validation_0-auc:0.70839
[7]	validation_0-auc:0.710084
[8]	validation_0-auc:0.712798
[9]	validation_0-auc:0.713794
[10]	validation_0-auc:0.715551
[11]	validation_0-auc:0.718352
[12]	validation_0-auc:0.721639
[13]	validation_0-auc:0.724157
[14]	validation_0-auc:0.726897
[15]	validation_0-auc:0.72826
[16]	validation_0-auc:0.729489
[17]	validation_0-auc:0.731814
[18]	validation_0-auc:0.734178
[19]	validation_0-auc:0.736313
[20]	validation_0-auc:0.737915
[21]	validation_0-auc:0.738592
[22]	validation_0-auc:0.739411
[23]	validation_0-auc:0.740481
[24]	validation_0-auc:0.741897
[25]	validation_0-auc:0.742635
[26]	validation_0-auc:0.74316
[27]	validation_0-auc:0.744308
[28]	validation_0-auc:0.744449
[29]	validation_0-auc:0.745409
[30]	validation_0-auc

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0.1,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=1000, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=11.447862694300518,
              seed=None, silent=None, subsample=0.5, verbosity=1)

In [ ]:
print("best score: {0}, best iteration: {1}, best ntree limit {2}".format(clf.best_score, clf.best_iteration, clf.best_ntree_limit))

best score: 0.777008, best iteration: 204, best ntree limit 205


#Best Metrics

In [ ]:
n_estimators = clf.best_ntree_limit
clf = XGBClassifier(n_estimators=n_estimators, objective='binary:logistic', seed=27, gamma=0.1, max_depth=4, subsample=0.8, nthread=4, colsample_bytree=0.8, min_child_weight=4,  scale_pos_weight=2)
clf.fit(X_train, y_train, eval_set=[(X_test, y_test)], eval_metric='auc', early_stopping_rounds=10)

[0]	validation_0-auc:0.693295
Will train until validation_0-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.700719
[2]	validation_0-auc:0.70923
[3]	validation_0-auc:0.71326
[4]	validation_0-auc:0.719422
[5]	validation_0-auc:0.723466
[6]	validation_0-auc:0.724168
[7]	validation_0-auc:0.72482
[8]	validation_0-auc:0.725757
[9]	validation_0-auc:0.726074
[10]	validation_0-auc:0.726654
[11]	validation_0-auc:0.728441
[12]	validation_0-auc:0.728905
[13]	validation_0-auc:0.730304
[14]	validation_0-auc:0.730067
[15]	validation_0-auc:0.731447
[16]	validation_0-auc:0.733094
[17]	validation_0-auc:0.733734
[18]	validation_0-auc:0.734048
[19]	validation_0-auc:0.735963
[20]	validation_0-auc:0.737659
[21]	validation_0-auc:0.738807
[22]	validation_0-auc:0.739625
[23]	validation_0-auc:0.74003
[24]	validation_0-auc:0.742082
[25]	validation_0-auc:0.743108
[26]	validation_0-auc:0.744706
[27]	validation_0-auc:0.745644
[28]	validation_0-auc:0.747046
[29]	validation_0-auc:0.747254
[30]	validation_0-au

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0.1,
              learning_rate=0.1, max_delta_step=0, max_depth=4,
              min_child_weight=4, missing=None, n_estimators=205, n_jobs=1,
              nthread=4, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=2, seed=27,
              silent=None, subsample=0.8, verbosity=1)

In [ ]:
# y_pred = xgb_model.predict(X_test)
y_pred = clf.predict(X_test)

#Results

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         0.0       0.93      0.99      0.96     42462
         1.0       0.43      0.13      0.19      3665

    accuracy                           0.92     46127
   macro avg       0.68      0.56      0.58     46127
weighted avg       0.89      0.92      0.90     46127



After using best_ntree_limit, we increase our ROC-AUC value from 0.777 to 0.7798. However, this result is remarkably lower than LightGBM. Lets continue to LightGBM.